# Conexion a Base de Datos
Una vez ya tenemos la informacion a analizar limpia, procedemos a hacer la conexion a la BD mediante SQL Server

## Importacion de libreria

In [ ]:
import pyodbc

## Desarrollo

In [ ]:
# Conexión a SQL Server
conn = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=localhost;"       # ej: DESKTOP-123\SQLEXPRESS
    "Database=Fuero_Federal;"       # la base
    "Trusted_Connection=yes;"   # o usa UID y PWD si es con usuario
)

cursor = conn.cursor()


In [ ]:
# Ejemplo de consulta
cursor.execute("SELECT TOP 5 * FROM Fuero_federal ORDER BY ENTIDAD")

for row in cursor.fetchall():
    print(row)

('2017', 'AGUASCALIENTES', 'CODIGO PENAL FEDERAL', 'CONTRA LA SALUD', 'PRODUCCION', 1, 'MAYO', 0, '2017-MAYO')
('2017', 'AGUASCALIENTES', 'CODIGO PENAL FEDERAL', 'CONTRA LA SALUD', 'PRODUCCION', 1, 'JULIO', 0, '2017-JULIO')
('2021', 'AGUASCALIENTES', 'OTRAS LEYES Y CODIGOS', 'OTRAS LEYES Y CODIGOS', 'CODIGO FISCAL DE LA FEDERACION (C.F.F.)', 1, 'OCTUBRE', 2, '2021-OCTUBRE')
('2018', 'AGUASCALIENTES', 'CODIGO PENAL FEDERAL', 'CONTRA LA SALUD', 'PRODUCCION', 1, 'ENERO', 0, '2018-ENERO')
('2015', 'AGUASCALIENTES', 'CODIGO PENAL FEDERAL', 'CONTRA LA SALUD', 'PRODUCCION', 1, 'JULIO', 0, '2015-JULIO')


Una vez corroboramos que la base este funcional y se tenga una conexion extable, procedere a reflejar los ejercicios solicitados pero ahora en SQL, para una comparacion de la informacion obtenida

### (a)  Número promedio de delitos agrupados por municipio y año-mes de ocurrencia de estos.

In [ ]:
query_a = """
SELECT 
    ENTIDAD AS municipio,
    AÑO_MES,
    AVG(DELITOS) AS promedio_delitos
FROM Fuero_federal
GROUP BY ENTIDAD, AÑO_MES
ORDER BY ENTIDAD, AÑO_MES;
"""

df_a = pd.read_sql(query_a, conn)
df_a.head(10)



C:\Users\RyanHz\AppData\Local\Temp\ipykernel_16356\3557139411.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_a = pd.read_sql(query_a, conn)


,municipio,AÑO_MES,promedio_delitos
0,AGUASCALIENTES,2012-ABRIL,3
1,AGUASCALIENTES,2012-AGOSTO,2
2,AGUASCALIENTES,2012-DICIEMBRE,1
3,AGUASCALIENTES,2012-ENERO,3
4,AGUASCALIENTES,2012-FEBRERO,3
5,AGUASCALIENTES,2012-JULIO,2
6,AGUASCALIENTES,2012-JUNIO,4
7,AGUASCALIENTES,2012-MARZO,4
8,AGUASCALIENTES,2012-MAYO,3
9,AGUASCALIENTES,2012-NOVIEMBRE,2


### (b)  El delito más común por municipio

In [ ]:
query_b = """
WITH ConteoDelitos AS (
    SELECT 
        ENTIDAD,
        TIPO AS delito,
        SUM(DELITOS) AS total_delitos,
        ROW_NUMBER() OVER (
            PARTITION BY ENTIDAD 
            ORDER BY SUM(DELITOS) DESC
        ) AS rn
    FROM Fuero_federal
    GROUP BY ENTIDAD, TIPO
)
SELECT 
    ENTIDAD,
    delito,
    total_delitos
FROM ConteoDelitos
WHERE rn = 1
ORDER BY ENTIDAD;
"""

df_b = pd.read_sql(query_b, conn)
df_b.head(10)


C:\Users\RyanHz\AppData\Local\Temp\ipykernel_16356\3865708296.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_b = pd.read_sql(query_b, conn)


,ENTIDAD,delito,total_delitos
0,AGUASCALIENTES,LEY FEDERAL DE ARMAS DE FUEGO Y EXPLOSIVOS (L....,1753
1,BAJA CALIFORNIA,LEY FEDERAL DE ARMAS DE FUEGO Y EXPLOSIVOS (L....,18130
2,BAJA CALIFORNIA SUR,PATRIMONIALES,2156
3,CAMPECHE,PATRIMONIALES,1859
4,CHIAPAS,PATRIMONIALES,5440
5,CHIHUAHUA,LEY FEDERAL DE ARMAS DE FUEGO Y EXPLOSIVOS (L....,10907
6,CIUDAD DE MEXICO,LEYES DE INSTITUCIONES DE CREDITO INVERSION FI...,46194
7,COAHUILA,PATRIMONIALES,4991
8,COLIMA,LEY FEDERAL DE ARMAS DE FUEGO Y EXPLOSIVOS (L....,2684
9,DURANGO,PATRIMONIALES,3451


### (c)  La temporada con más crímenes en México

In [ ]:
query_c = """
SELECT 
    CASE 
        WHEN MES IN ('MARZO','ABRIL','MAYO') THEN 'PRIMAVERA'
        WHEN MES IN ('JUNIO','JULIO','AGOSTO') THEN 'VERANO'
        WHEN MES IN ('SEPTIEMBRE','OCTUBRE','NOVIEMBRE') THEN 'OTOÑO'
        WHEN MES IN ('DICIEMBRE','ENERO','FEBRERO') THEN 'INVIERNO'
    END AS temporada,
    SUM(DELITOS) AS total_delitos
FROM Fuero_federal
GROUP BY 
    CASE 
        WHEN MES IN ('MARZO','ABRIL','MAYO') THEN 'PRIMAVERA'
        WHEN MES IN ('JUNIO','JULIO','AGOSTO') THEN 'VERANO'
        WHEN MES IN ('SEPTIEMBRE','OCTUBRE','NOVIEMBRE') THEN 'OTOÑO'
        WHEN MES IN ('DICIEMBRE','ENERO','FEBRERO') THEN 'INVIERNO'
    END
ORDER BY total_delitos DESC;
"""

df_c = pd.read_sql(query_c, conn)
df_c


C:\Users\RyanHz\AppData\Local\Temp\ipykernel_16356\3944615053.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_c = pd.read_sql(query_c, conn)


,temporada,total_delitos
0,VERANO,334840
1,PRIMAVERA,331463
2,OTOÑO,309647
3,INVIERNO,297062


###  (d)  El número promedio de delitos agrupados por tipo de delito y año y mes en la ciudad de México ordenados de mayor a menor

In [ ]:
query_d = """
SELECT 
    TIPO,
    AÑO,
    MES,
    AVG(DELITOS) AS promedio_delitos
FROM Fuero_federal
WHERE ENTIDAD = 'CIUDAD DE MEXICO'
GROUP BY TIPO, AÑO, MES
ORDER BY promedio_delitos DESC;
"""

df_d = pd.read_sql(query_d, conn)
df_d.head(10)


C:\Users\RyanHz\AppData\Local\Temp\ipykernel_16356\1731659320.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_d = pd.read_sql(query_d, conn)


,TIPO,AÑO,MES,promedio_delitos
0,LEYES DE INSTITUCIONES DE CREDITO INVERSION FI...,2018,ABRIL,928
1,LEYES DE INSTITUCIONES DE CREDITO INVERSION FI...,2018,MAYO,847
2,LEYES DE INSTITUCIONES DE CREDITO INVERSION FI...,2018,AGOSTO,825
3,LEYES DE INSTITUCIONES DE CREDITO INVERSION FI...,2018,JULIO,763
4,LEYES DE INSTITUCIONES DE CREDITO INVERSION FI...,2018,ENERO,757
5,LEYES DE INSTITUCIONES DE CREDITO INVERSION FI...,2018,OCTUBRE,754
6,LEYES DE INSTITUCIONES DE CREDITO INVERSION FI...,2018,NOVIEMBRE,739
7,LEYES DE INSTITUCIONES DE CREDITO INVERSION FI...,2017,JUNIO,719
8,LEYES DE INSTITUCIONES DE CREDITO INVERSION FI...,2018,JUNIO,711
9,LEYES DE INSTITUCIONES DE CREDITO INVERSION FI...,2019,FEBRERO,682


### Cierre de recursos

In [ ]:
# Cerrar recursos
cursor.close()
conn.close()

---